# Tensorflow Hub Model with Five Percent Data Augmenetation (Top 10 Diagnosis)

In [1]:
import pickle, random
import numpy as np

In [2]:
import pandas as pd

In [3]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [4]:
winogender_sentences = "all_sentences.tsv"

In [5]:
winogender_occupations = "occupations-stats.tsv"

In [6]:
import csv

wino_occupations = []

In [7]:
with open(winogender_occupations, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        wino_occupations.append(lines["occupation"])

In [8]:
# wino_occupations

In [9]:
len(wino_occupations)

60

In [10]:
len(set(wino_occupations))

60

### Load Text Classification Hub Model

In [11]:
import tensorflow as tf

In [12]:
rand_range = range(1, 11)

In [13]:
# my_model directory
# !ls ../saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls ../saved_model/five_percent_models/

my_model_txt_classifier_hub_with_five_percent_extra_data_rand_1
my_model_txt_classifier_hub_with_five_percent_extra_data_rand_10
my_model_txt_classifier_hub_with_five_percent_extra_data_rand_2
my_model_txt_classifier_hub_with_five_percent_extra_data_rand_3
my_model_txt_classifier_hub_with_five_percent_extra_data_rand_4
my_model_txt_classifier_hub_with_five_percent_extra_data_rand_5
my_model_txt_classifier_hub_with_five_percent_extra_data_rand_6
my_model_txt_classifier_hub_with_five_percent_extra_data_rand_7
my_model_txt_classifier_hub_with_five_percent_extra_data_rand_8
my_model_txt_classifier_hub_with_five_percent_extra_data_rand_9


In [14]:
!ls ../saved_model/six_percent_models/my_model_txt_classifier_hub_with_five_percent_extra_data_rand_1

ls: ../saved_model/six_percent_models/my_model_txt_classifier_hub_with_five_percent_extra_data_rand_1: No such file or directory


In [15]:
percent_mode = "five"
path_start = "../saved_model/" + str(percent_mode) + "_percent_models/my_model_txt_classifier_hub_with_" + str(percent_mode) + "_percent_extra_data_rand_"
path_start

'../saved_model/five_percent_models/my_model_txt_classifier_hub_with_five_percent_extra_data_rand_'

## Test on Repaired Input Tokens

### A. Direct gender noun comparisons (e.g. My boyfriend/My girlfriend)

In [16]:
mode = "direct-gender-noun"

In [17]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [18]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [19]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [20]:
noun_error = {**noun_error1, **noun_error2}

In [21]:
# noun_error 

In [22]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [23]:
noun_error1

['my aunt',
 'my daughter',
 'my mother',
 'my wife',
 'this girl',
 'my sister',
 'this woman',
 'my girlfriend']

In [24]:
noun_error2

['my uncle',
 'my son',
 'my father',
 'my brother',
 'my husband',
 'this boy',
 'this man',
 'my boyfriend']

In [25]:
noun_error = get_sorted_dict(noun_error)

In [26]:
noun_error = list(noun_error)

In [27]:
input1, input2, input3 = [],[], []

In [28]:
input_pairs, input_truple = [], []

In [29]:
occupation_list = []

In [30]:
i = 0
tmp, tmp2 ="", ""

In [31]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [32]:
f_input, m_input, neut_input = "", "", ""

In [33]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [34]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [35]:
# fairness_errors = []
# all_inputs = []
# count = 0

In [36]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [37]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [38]:
len(all_inputs)

3000

In [39]:
len(fairness_errors)

270

In [40]:
len(fairness_errors_all_rands)

10

In [41]:
fairness_errors_all_rands

[93, 179, 309, 185, 234, 305, 131, 76, 206, 270]

In [42]:
sum(fairness_errors_all_rands)

1988

In [43]:
sum(fairness_errors_all_rands)/10

198.8

In [44]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [45]:
sum(all_rand_inputs)

30000

In [46]:
sum(all_rand_inputs)/10

3000.0

In [47]:
A_errors = sum(fairness_errors_all_rands)

In [48]:
A_inputs = sum(all_rand_inputs)

### B. Random gender noun comparisons (e.g. My boyfriend/My mother)

In [49]:
mode = "random-gender-noun"

In [50]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [51]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [52]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [53]:
noun_error = {**noun_error1, **noun_error2}

In [54]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [55]:
noun_error1

['my aunt',
 'my girlfriend',
 'my daughter',
 'my wife',
 'my mother',
 'this woman',
 'this girl',
 'my sister',
 'my mom']

In [56]:
noun_error2

['my brother',
 'my father',
 'this boy',
 'my boyfriend',
 'my uncle',
 'my husband',
 'this man',
 'my dad',
 'my son']

In [57]:
noun_error = get_sorted_dict(noun_error)

In [58]:
noun_error = list(noun_error)

In [59]:
input1, input2, input3 = [],[], []

In [60]:
input_pairs, input_truple = [], []

In [61]:
occupation_list = []

In [62]:
i = 0
tmp, tmp2 ="", ""

In [63]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [64]:
f_input, m_input, neut_input = "", "", ""

In [65]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = ""
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = ""
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [66]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [67]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [68]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [69]:
len(all_inputs)

3000

In [70]:
len(fairness_errors)

200

In [71]:
len(fairness_errors_all_rands)

10

In [72]:
fairness_errors_all_rands

[91, 105, 220, 225, 138, 205, 113, 72, 118, 200]

In [73]:
sum(fairness_errors_all_rands)

1487

In [74]:
sum(fairness_errors_all_rands)/10

148.7

In [75]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [76]:
sum(all_rand_inputs)

30000

In [77]:
sum(all_rand_inputs)/10

3000.0

In [78]:
B_errors = sum(fairness_errors_all_rands)

In [79]:
B_inputs = sum(all_rand_inputs)

### C. Test for Indirect Gender Bias, i.e. Occupational Bias

In [80]:
mode = "gender-occupation-noun"

In [81]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [82]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [83]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [84]:
noun_error = {**noun_error1, **noun_error2}

In [85]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [86]:
noun_error1

['the writer',
 'the teacher',
 'the cleaner',
 'the auditor',
 'the counselor',
 'the nurse',
 'the the paralegal',
 'the editor',
 'the tailor',
 'the clerk']

In [87]:
noun_error2

['the cook',
 'the analyst',
 'the engineer',
 'the mechanic',
 'the technician',
 'the supervisor',
 'the salesperson',
 'the farmer',
 'the chief',
 'the lawyer']

In [88]:
noun_error = get_sorted_dict(noun_error)

In [89]:
noun_error = list(noun_error)

In [90]:
input1, input2, input3 = [],[], []

In [91]:
input_pairs, input_truple = [], []

In [92]:
occupation_list = []

In [93]:
i = 0
tmp, tmp2 ="", ""

In [94]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [95]:
f_input, m_input, neut_input = "", "", ""

In [96]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1

In [97]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [98]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [99]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [100]:
len(all_inputs)

3000

In [101]:
len(fairness_errors)

265

In [102]:
len(fairness_errors_all_rands)

10

In [103]:
fairness_errors_all_rands

[262, 201, 225, 153, 219, 378, 256, 139, 210, 265]

In [104]:
sum(fairness_errors_all_rands)

2308

In [105]:
sum(fairness_errors_all_rands)/10

230.8

In [106]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [107]:
sum(all_rand_inputs)

30000

In [108]:
sum(all_rand_inputs)/10

3000.0

In [109]:
c_errors = sum(fairness_errors_all_rands)

In [110]:
C_inputs = sum(all_rand_inputs)

### D. Test for   for Indirect Gender Bias, i.e. Name Bias

In [111]:
mode = "gender-name-noun"

In [112]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [113]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [114]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [115]:
noun_error = {**noun_error1, **noun_error2}

In [116]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [117]:
noun_error1

['Deborah',
 'Sharon',
 'Kimberly',
 'Carol',
 'Michelle',
 'Karen',
 'Sandra',
 'Laura',
 'Nancy',
 'Barbara']

In [118]:
noun_error2

['Christopher',
 'Joshua',
 'Thomas',
 'James',
 'Timothy',
 'Mark',
 'Robert ',
 'George',
 'Edward',
 'Anthony']

In [119]:
noun_error = get_sorted_dict(noun_error)

In [120]:
noun_error = list(noun_error)

In [121]:
input1, input2, input3 = [],[], []

In [122]:
input_pairs, input_truple = [], []

In [123]:
occupation_list = []

In [124]:
i = 0
tmp, tmp2 ="", ""

In [125]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [126]:
f_input, m_input, neut_input = "", "", ""

In [127]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" 
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1

In [128]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [129]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [130]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [131]:
len(all_inputs)

3000

In [132]:
len(fairness_errors)

139

In [133]:
len(fairness_errors_all_rands)

10

In [134]:
fairness_errors_all_rands

[328, 268, 349, 223, 215, 436, 171, 112, 146, 139]

In [135]:
sum(fairness_errors_all_rands)

2387

In [136]:
sum(fairness_errors_all_rands)/10

238.7

In [137]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [138]:
sum(all_rand_inputs)

30000

In [139]:
sum(all_rand_inputs)/10

3000.0

In [140]:
D_errors = sum(fairness_errors_all_rands)

In [141]:
D_inputs = sum(all_rand_inputs)

### E. Test for   for Indirect Racial Bias, i.e. Name Bias

In [142]:
mode = "racial-name-noun"

In [143]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [144]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [145]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [146]:
noun_error = {**noun_error1, **noun_error2}

In [147]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [148]:
noun_error1

['Josh',
 'Melanie',
 'Justin',
 'Stephanie',
 'Harry',
 'Jack',
 'Katie',
 'Nancy',
 'Amanda',
 'Ellen']

In [149]:
noun_error2

['Malik',
 'Lamar',
 'Jasmine',
 'Jamel',
 'Nichelle',
 'Latisha',
 'Darnell',
 'Leroy',
 'Alonzo',
 'Shereen']

In [150]:
noun_error = get_sorted_dict(noun_error)

In [151]:
noun_error = list(noun_error)

In [152]:
input1, input2, input3 = [],[], []

In [153]:
input_pairs, input_truple = [], []

In [154]:
occupation_list = []

In [155]:
i = 0
tmp, tmp2 ="", ""

In [156]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [157]:
f_input, m_input, neut_input = "", "", ""

In [158]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = ""
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [159]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [160]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [161]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [162]:
len(all_inputs)

3000

In [163]:
len(fairness_errors)

192

In [164]:
len(fairness_errors_all_rands)

10

In [165]:
fairness_errors_all_rands

[193, 181, 293, 168, 366, 302, 188, 87, 184, 192]

In [166]:
sum(fairness_errors_all_rands)

2154

In [167]:
sum(fairness_errors_all_rands)/10

215.4

In [168]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [169]:
sum(all_rand_inputs)

30000

In [170]:
sum(all_rand_inputs)/10

3000.0

In [171]:
E_errors = sum(fairness_errors_all_rands)

In [172]:
E_inputs = sum(all_rand_inputs)

### F. Test for Neutral (Sentiment) Sentences

In [173]:
mode = "neutral-sentiments"

In [174]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [175]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [176]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [177]:
noun_error = {**noun_error1, **noun_error2}

In [178]:
# noun_error 

In [179]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [180]:
noun_error1

['my aunt',
 'my daughter',
 'my wife',
 'my mother',
 'my girlfriend',
 'this girl',
 'this woman',
 'my sister',
 'my mom',
 'the writer']

In [181]:
noun_error2

['my son',
 'my uncle',
 'my father',
 'my husband',
 'my brother',
 'this boy',
 'this man',
 'my boyfriend',
 'my dad',
 'the cook']

In [182]:
noun_error = get_sorted_dict(noun_error)

In [183]:
noun_error = list(noun_error)

In [184]:
input1, input2, input3 = [],[], []

In [185]:
input_pairs, input_truple = [], []

In [186]:
occupation_list = []

In [187]:
i = 0
tmp, tmp2 ="", ""

In [188]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [189]:
f_input, m_input, neut_input = "", "", ""

In [190]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" 
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [191]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [192]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [193]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [194]:
len(all_inputs)

3000

In [195]:
len(fairness_errors)

263

In [196]:
len(fairness_errors_all_rands)

10

In [197]:
fairness_errors_all_rands

[101, 172, 192, 212, 199, 295, 209, 116, 178, 263]

In [198]:
sum(fairness_errors_all_rands)

1937

In [199]:
sum(fairness_errors_all_rands)/10

193.7

In [200]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [201]:
sum(all_rand_inputs)

30000

In [202]:
sum(all_rand_inputs)/10

3000.0

In [203]:
F_errors = sum(fairness_errors_all_rands)

In [204]:
F_inputs = sum(all_rand_inputs)

In [207]:
all_inputs = A_inputs + B_inputs + C_inputs + D_inputs + E_inputs + F_inputs

In [208]:
all_errors = A_errors + B_errors + c_errors + D_errors + E_errors + F_errors

In [209]:
all_inputs

180000

In [210]:
all_errors

12261

In [211]:
all_inputs/6

30000.0

In [212]:
all_errors/6

2043.5

In [213]:
all_errors_list = [A_errors, B_errors, c_errors, D_errors, E_errors, F_errors]
print("all_errors_list: ", all_errors_list)
all_inputs_list = [A_inputs, B_inputs, C_inputs, D_inputs, E_inputs, F_inputs]
print("all_inputs_list: ", all_inputs_list)

all_errors_list:  [1988, 1487, 2308, 2387, 2154, 1937]
all_inputs_list:  [30000, 30000, 30000, 30000, 30000, 30000]
